In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import dateutil.parser as parser
import numpy as np
from matplotlib import pyplot as plt

In [4]:
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield eval(l)

In [13]:
count = 0
train_data = []
test_data = []

usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for d in parse("./lthing_data/reviews.json"):
    # handle normal review (majority) and 'abuse' or 'not a review' flag
    if not d['flags']:
        d['flags'] = 0
    else:
        d['flags'] = 1
    
    # train and test
    if count < 1500000:
        train_data.append(d)
        usersPerItem[d['work']].add(d['user'])
        itemsPerUser[d['user']].add(d['work'])
    else:
        test_data.append(d)
    count += 1

print(len(test_data))
test_data[1]

207070


{'work': '9910140',
 'flags': 1,
 'unixtime': 1283472000,
 'nhelpful': 0,
 'time': 'Sep 3, 2010',
 'comment': 'REALLY good book!',
 'user': 'Bailette25'}

# baseline model

### A relevant simple baseline that can be used for comparisonis predicting "would read" based on the popularity of thebook, considering a user is likely to read a book if that bookis popular in general.

In [15]:
len(usersPerItem)
# len(itemsPerUser)

464649

In [16]:
len(itemsPerUser)

79849

In [36]:
mostpopular = {item for item in usersPerItem if len(usersPerItem[item]) > 1}
len(mostpopular)

154882

### About one third of books have more than 1 reader, and we consider them as popular books

In [37]:
# Construct baseline test data
allbooks = list(usersPerItem.keys())
Xtest = []
Ytest = []
for d in test_data:
    u,i = d['user'], d['work']
    Xtest.append((u,i))
    Ytest.append(1)
    i2 = random.choice(allbooks)
    while u in itemsPerUser and i2 in itemsPerUser[u]:
        # sampled another book this user has read
        i2 = random.choice(allbooks)
    Xtest.append((u,i2))
    Ytest.append(0)
    
len(Xtest)

414140

In [38]:
Ypred = [i in mostpopular for u,i in Xtest]
len(Ypred)

414140

In [39]:
sum(a==b for a,b in zip(Ytest,Ypred)) / len(Ypred)

0.6920437533201332